# Fluent Python
https://github.com/fluentpython/example-code

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np

def time_check(func):
    def decorated():
        import time
        start = time.time()
        func()
        print("---{}s seconds---".format(time.time()-start_time))
    return decorated# Fluent Python

## CHAPTER 5 First-Class Functions

**first-class object**
- Created at runtime
- Assigned to a variable or element in a data structure
- Passed as an argument to a function
- Returned as the result of a function

Integers, strings, and dictionaries are other examples of first-class objects in Python

### Treating a Function Like an Object

In [5]:
def factorial(n):
    '''returns n!'''
    return 1 if n < 2 else n * factorial(n-1)

In [6]:
factorial.__doc__
type(factorial)

'returns n!'

function

In [7]:
fact = factorial
list(map(fact, range(11)))

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

### Higher-Order Functions
A function that takes a function as argument or returns a function as the result is a
higher-order function

sorted(), map(), filter(), reduce()

In [8]:
list(map(fact, range(6)))

[1, 1, 2, 6, 24, 120]

In [9]:
list(map(factorial, filter(lambda n: n%2, range(6))))
[factorial(n) for n in range(6) if n % 2]

[1, 6, 120]

[1, 6, 120]

In [10]:
from functools import reduce
from operator import add
reduce(add, range(100))
sum(range(100)) #가독성 성능면에서 더 낫다.

4950

4950

### Anonymous Functions 

lambda

**Lundh’s lambda Refactoring Recipe**
1. Write a comment explaining what the heck that lambda does.
2. Study the comment for a while, and think of a name that captures the essence of the comment.
3. Convert the lambda to a def statement, using that name.
4. Remove the comment.

[the Functional Programming HOWTO](https://docs.python.org/3/howto/functional.html)

### The Seven Flavors of Callable Objects

- User-defined functions
    Created with def statements or lambda expressions.
- Built-in functions
    A function implemented in C (for CPython), like len or time.strftime .
- Built-in methods
    Methods implemented in C, like dict.get .
- Methods
    Functions defined in the body of a class.
- Classes
    When invoked, a class runs its \__new__ method to create an instance, then \__init__ to initialize it, and finally the instance is returned to the caller. 
    new 연산자가 따로 없음으로 클래스는 함수호출과 동일하다.
- Class instances
    If a class defines a \__call\__ method, then its instances may be invoked as functions.
- Generator functions
    Functions or methods that use the yield keyword. When called, generator func‐tions return a generator object.

\__call\__() 메서드를 구현하면 모든 파이썬 객체가 함수처럼 동작할수 있다.

### Function Introspection

In [15]:
class c: pass
obj = c()
def func(): pass
sorted(set(dir(func))-set(dir(obj))) # 객체에는 존재하지 않는 함수 속성 나열 
sorted(set(dir(obj))-set(dir(func)))

['__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__']

['__weakref__']

### Retrieving Information About Parameters

- \__defaults\__ : the default values of positional and keyword arguments. 

- \__kwdefaults\__ : The defaults for keyword-only arguments. 

- \__code\__ : The names of the arguments, which is a reference to a code object with many attributes of its own.


In [27]:
def factorial(i, n=10):
    '''returns n!'''
    return 1 if n < 2 else n * factorial(n, n-1)

In [28]:
factorial.__defaults__
factorial.__code__
factorial.__code__.co_varnames
factorial.__code__.co_argcount

(10,)

<code object factorial at 0x7f2d1aa0bd20, file "<ipython-input-27-c08eb3741051>", line 1>

('i', 'n')

2

inspect 모듈 사용

In [29]:
from inspect import signature
sig = signature(factorial)
sig
str(sig)

<Signature (i, n=10)>

'(i, n=10)'

In [30]:
for name, param in sig.parameters.items():
    print(param.kind, ':', name, '=', param.default)

POSITIONAL_OR_KEYWORD : i = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : n = 10


param.kind : POSITIONAL_OR_KEYWORD, VAR_POSITIONAL, VAR_KEYWORD, KEYWORD_ONLY, POSITIONAL_ONLY

bind method

In [37]:
def tag(name, *content, cls=None, **attrs):
    """Generate one or more HTML tags"""
    return '\n'.join('<%s%s>%s</%s>' % (name, attr_str, c, name) for c in content)

In [38]:
sig = signature(tag)
my_tag = {'name': 'img', 'title': 'Sunset Boulevard','src': 'sunset.jpg', 'cls': 'framed'}
bound_args = sig.bind(**my_tag)
bound_args

<BoundArguments (name='img', cls='framed', attrs={'title': 'Sunset Boulevard', 'src': 'sunset.jpg'})>

In [39]:
for name, value in bound_args.arguments.items():
    print(name, '=', value)

name = img
cls = framed
attrs = {'title': 'Sunset Boulevard', 'src': 'sunset.jpg'}


### Function Annotations

In [40]:
def clip(text:str, max_len:'int > 0'=80) -> str:
    """Return text clipped at the last space before or after max_len
    """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
        if space_after >= 0:
            end = space_after
    
    if end is None: # no spaces were found
            end = len(text)
    return text[:end].rstrip()

In [41]:
clip.__annotations__

{'text': str, 'max_len': 'int > 0', 'return': str}

annotations 는 정의만 되어 있을뿐 검증, 단속하지 않는다. 

annotation 추출하는 방법

In [43]:
sig = signature(clip)
sig.return_annotation

str

In [44]:
for param in sig.parameters.values():
    note = repr(param.annotation).ljust(13)
    print(note, ':', param.name, '=', param.default)

<class 'str'> : text = <class 'inspect._empty'>
'int > 0'     : max_len = 80


### Packages for Functional Programming

In [46]:
from functools import reduce
from operator import mul

def fact(n):
    # return reduce(lambda a, b: a*b, range(1, n+1))
    return reduce(mul, range(1, n+1))

fact(5)

120

In [47]:
metro_data = [
('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

* itemgetter()

In [48]:
from operator import itemgetter
for city in sorted(metro_data, key=itemgetter(1)):
    print(city)

('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))


In [49]:
cc_name = itemgetter(1, 0)
for city in metro_data:
    print(cc_name(city))

('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'Sao Paulo')


* attrgetter()

In [53]:
from collections import namedtuple
LatLong = namedtuple('LatLong', 'lat long')
Metropolis = namedtuple('Metropolis', 'name cc pop coord')
metro_areas = [Metropolis(name, cc, pop, LatLong(lat, long)) 
    for name, cc, pop, (lat, long) in metro_data]

metro_areas[0]

from operator import attrgetter
name_lat = attrgetter('name', 'coord.lat')
for city in sorted(metro_areas, key=attrgetter('coord.lat')): 
    print(name_lat(city))

Metropolis(name='Tokyo', cc='JP', pop=36.933, coord=LatLong(lat=35.689722, long=139.691667))

('Sao Paulo', -23.547778)
('Mexico City', 19.433333)
('Delhi NCR', 28.613889)
('Tokyo', 35.689722)
('New York-Newark', 40.808611)


* methodcaller() 어떤 메소드를 호출해 주는 역할

In [55]:
from operator import methodcaller
s = 'The time has come'
upcase = methodcaller('upper')
upcase(s)

'THE TIME HAS COME'

In [56]:
hiphenate = methodcaller('replace', ' ', '-')
hiphenate(s)

'The-time-has-come'

* functools.partial()

In [57]:
from operator import mul
from functools import partial
triple = partial(mul, 3) # mul 함수는 2개의 인수를 필요로 하지만 3 을 고정함으로 인수 1개만 있으면된다.
triple(7)

21